In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier as xgb
from sklearn.naive_bayes import GaussianNB

In [ ]:
df = pd.read_csv('/heart-disease-UCI.csv')
print(df.describe)

In [ ]:
print(df.info())


In [ ]:
df.target.value_counts(normalize=True)

In [ ]:

numeric_cols = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
                'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']


X = df[numeric_cols]
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=numeric_cols).reset_index(drop=True)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=numeric_cols).reset_index(drop=True)

print(X_train_scaled_df.head())


In [ ]:
train_final = pd.concat([X_train_scaled_df, y_train.reset_index(drop=True)], axis=1)
test_final = pd.concat([X_test_scaled_df, y_test.reset_index(drop=True)], axis=1)

X_train = train_final.drop('target', axis=1)
y_train = train_final['target']

X_test = test_final.drop('target', axis=1)
y_test = test_final['target']


In [ ]:
model = SVC(kernel='poly', random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("\n🔷 SVM ------------------")
print("✅ Accuracy:", round(accuracy_score(y_test, y_pred), 4))
print("📊 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("📝 Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
log_model = LogisticRegression(random_state=42,max_iter=100)
log_model.fit(X_train,y_train)
y_pred_log = log_model.predict(X_test)

print("\n🔷 Logistic Regression ------------------")
print("✅ Accuracy:", accuracy_score(y_test, y_pred_log))
print("📊 Confusion Matrix:\n", confusion_matrix(y_test, y_pred_log))
print("📝 Classification Report:\n", classification_report(y_test, y_pred_log))

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='euclidean')
knn.fit(X_train,y_train)
y_pred_knn = knn.predict(X_test)

print("\n🔷 K-Nearest Neighbors ------------------")
print("✅ Accuracy:", accuracy_score(y_test, y_pred_knn))
print("📊 Confusion Matrix:\n", confusion_matrix(y_test, y_pred_knn))
print("📝 Classification Report:\n", classification_report(y_test, y_pred_knn))

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(100,),activation='relu',solver='sgd',early_stopping=True,max_iter=500,random_state=42)
mlp.fit(X_train,y_train)
y_pred_mlp = mlp.predict(X_test)


print("\n🔷 MLPClassifier (Neural Network) ------------------")
print("✅ Accuracy:", accuracy_score(y_test, y_pred_mlp))
print("📊 Confusion Matrix:\n", confusion_matrix(y_test, y_pred_mlp))
print("📝 Classification Report:\n", classification_report(y_test, y_pred_mlp))

In [ ]:
dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)
y_pred_tree =dtree.predict(X_test)

print("\n🔷 Decision Tree ------------------")
print("✅ Accuracy:", accuracy_score(y_test, y_pred_tree))
print("📊 Confusion Matrix:\n", confusion_matrix(y_test, y_pred_tree))
print("📝 Classification Report:\n", classification_report(y_test, y_pred_tree))



In [ ]:
rf = RandomForestClassifier(n_estimators=100,max_depth=None,random_state=42)
rf.fit(X_train,y_train)
y_pred_rf = rf.predict(X_test)

print("\n🔷 Random Forest Classifier ------------------")
print("✅ Accuracy:", accuracy_score(y_test, y_pred_rf))
print("📊 Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))
print("📝 Classification Report:\n", classification_report(y_test, y_pred_rf))